In [1]:
import pandas as pd
from skbio.stats.ordination import OrdinationResults
# Für später wahrscheinlich
# from qiime2.plugins import emperor
# from qiime2.plugins import diversity


1: Dataframe erstellen mit metadata.tsv datei

1.1: Einlesen des Metadata file

In [ ]:
df1 = pd.read_csv('/home/sven/Desktop/testing/testkurth/meta_plate.tsv' , sep='\t')
#Path ist noch lokal, work in progress
df2 = df1

1.2 Falls Metadata file einen anderen namen als well_id haben sollte dann namen finden und umbenennen, anschließend checken ob well_id teil der indizes ist.
okay also problem, sobald die metadata file bereits den spaltennamen "well_id" enthält will die conversion nicht mehr funktionieren. Auch der untere Code bekommt es (noch) nicht hin.

In [3]:
werte = ["A1", "A2"] # Das hier muss noch unbedingt mit regex gemacht werden für den wahrscheinlichen fall dass auf der 
#plate die ersten beiden wells nicht benutzt werden.

# Passende Spalte finden und umbenennen
if "well_id" not in df2.columns:
    for spalte in df2.columns:
        if all(wert in df2[spalte].values for wert in werte):
            print(f"beide Werte in spalte: {spalte}")
else:
    1
   
df2 = df2.rename(columns={spalte: "well_id"}) 
# testing: nachschauen ob es geklappt hat
print(df2.columns)



beide Werte in spalte: sample_well_y
Index(['sample_name', 'active_disease_symptoms', 'age', 'age_unit',
       'allergies', 'anti_nuclear_antibody', 'anti_nuclear_antibody_positiv',
       'antibiotics_last_6_months', 'biopsy', 'body_product',
       ...
       'platform', 'primer', 'qiita_prep_id', 'run_center', 'run_date',
       'run_prefix', 'sequencing_meth', 'target_gene', 'target_subfragment',
       'well_id'],
      dtype='object', length=126)


Faxen (also wirklich nur faxen, es gibt eine so viel bessere idee und die werde ich dann auch machen allerdings funktioniert das hier (leider gottes) und es erlaubt mir die weiteren schritte zu gehen.)

In [4]:

df2["well_id"] = df2["well_id"].str.replace("A", "8, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("B", "7, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("C", "6, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("D", "5, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("E", "4, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("F", "3, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("G", "2, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("H", "1, ", regex=False)

1.3: Das große Metadata file verkleinern in die für die Ordination interessanten Informationen(sample_name, well_id) herausfiltern

In [5]:
tempSamples = pd.concat([
    df2["sample_name"],
    df2["well_id"]],
    axis =1 
    )

# Sehr wichtig: Es kann passieren dass in sample_name mehr rows 
# existieren als well_id werte, dann packt der code fälschlicherweise NaN values auf die row.
# Hiermit versucht man dem ein bisschen entgegenzuwirken und so far funktioniert es
x = 0
for val in tempSamples["well_id"].values:
    x += 1
    if type(val) is not str:
        print(type(val)) # NaN ist n float
        print(tempSamples.sample_name.values[x-1])
        tempSamples = tempSamples.drop(x - 1)

# testing: worked?
print(tempSamples.head)


<bound method NDFrame.head of              sample_name well_id
0           14813.POAF61    3, 2
1          14813.POASW56    2, 1
2          14813.POASW59    8, 1
3          14813.POASW61    5, 2
4          14813.POASW63    1, 3
5          14813.POASW64    3, 3
6          14813.POASW65    6, 3
7          14813.POASW66    8, 3
8          14813.POASW69    2, 4
9          14813.POASW71    5, 4
10         14813.POASW72    7, 4
11         14813.POASW80    6, 5
12         14813.POASW81    8, 5
13         14813.POASW83    1, 6
14         14813.POASZ56    1, 1
15         14813.POASZ57    3, 1
16         14813.POASZ59    7, 1
17         14813.POASZ61    4, 2
18         14813.POASZ63    8, 2
19         14813.POASZ64    2, 3
20         14813.POASZ65    5, 3
21         14813.POASZ66    7, 3
22         14813.POASZ69    1, 4
23         14813.POASZ70    3, 4
24         14813.POASZ71    4, 4
25         14813.POASZ72    6, 4
26         14813.POASZ81    7, 5
27         14813.POASZ85    2, 6
28         14

Ab hier, Chatgpt war die inspiration, nach kurzem googlen hab ich herausgefunden wie ich das für mich umschreiben muss. 

1.4: Die zusammengesetzten Zahlen der well_id trennen und in zwei spalten (Axis1, Axis2) trennen. Zum ende hin die nummern als float ausgeben lassen da es mit str vermutlich noch weniger funktioniert. 

In [6]:
#split
tempSamples[["Axis1", "Axis2"]] = tempSamples["well_id"].str.split(',', n=1, expand=True)
#datentypen fest deklarieren
# dabei schon die Achsen tauschen, da:
#     x -------->
#         0 1 2 3 ...
# y   a 
# |   b
# |   c
# |
# Im Originalzustand enthält Axis1 die a,b,c... werte und Axis2 die 0-12 werte, die ordination datei nimmt als erste achse halt x


#"Drehen" der beiden Achsen
temp = tempSamples["Axis1"]
tempSamples["Axis1"] = tempSamples["Axis2"].astype(float)
tempSamples["Axis2"] = temp.astype(float)

#finalen dataframe erstellen aus welchem auch die ordination am ende entsteht
samples = pd.DataFrame(
    data=tempSamples[["Axis1", "Axis2"]].values,
    index=tempSamples["sample_name"],
    columns=["Axis1", "Axis2"]
)
#test
print(samples.head(7))

               Axis1  Axis2
sample_name                
14813.POAF61     2.0    3.0
14813.POASW56    1.0    2.0
14813.POASW59    1.0    8.0
14813.POASW61    2.0    5.0
14813.POASW63    3.0    1.0
14813.POASW64    3.0    3.0
14813.POASW65    3.0    6.0


2.1: Eigenvalues und proportion_explained. Einfach fantasiewerte einfügen

In [7]:
eigvals = pd.Series([0.5, 0.3])
proportion_explained = pd.Series([0.5758, 0.4242])


2.2: Erstellung der Ordination und export als ordination.txt

In [8]:
ordination = OrdinationResults(
    "Manual",
    "Parsed well ID ordination",
    eigvals,
    samples,
    proportion_explained = proportion_explained
)


In [9]:
with open("ordination.txt", "w") as f:
    
    ordination.write(f, format="ordination")

Nächster Teil: mit Python automatisch einen Path und Ordner für das alles erstellen und dann darüber automatisch die .qzv zu erstellen.
Probleme die ich immernoch habe:    
- Der Code kann nur eine plate erstellen(falls mehr koordinaten als plätze auf wells, und dann auch noch gleich, dann bomba)
    - vielleicht dem Programm sagen, sobald mehr als 96 wellids da sind dann pack die in einen dateframe und geh weitere 96 schritte AUßER wir haben etabliert es handelt sich um eine 384x plate
    - ODER: bei mehreren gleichen well_ids nach irgendeinem Alleinstellungsmerkmal(krank,gesund) filtern und danach mehrere
    plates erzeugen.
- 384x plates sind noch nicht möglich (probably easiest to do)
    - Vielleicht irgendwie es schaffen, dem code zu sagen wie viele wells in einem Metafile stecken, demnach in mehrere Dataframes unterteilen
    - in etwa so: for vals in df2["Axis1"] if df2["Axis1"].values >=13 && df2["Axis2"].values >=9 dann 384 well plate
- Qiime2 befehle Integration in den Code, skip the Terminal